### T&uuml;rme von Hanoi, Callback Variante  

Der Controller sollte sich nur um das &Uuml;bermitteln eines Befehls an die Game-Komponente k&uuml;mmern m&uuml;ssen, nicht auch noch um die Darstellung. Zu diesem Zweck kommuniziert die
Game-Komponente via Callbacks mit der View:

- Je nach Tastatureingabe ruft der Controller die Funktion `new_game()` oder `move_disk(i, j)` der Game-Komponente auf.  
- Nach getaner Arbeit rufen dann `new_game` bez. `move_disk` das von der View registrierte Callback mit den Argumenten `event` und `data` auf, was zur Darstellung des Spielzustandes f&uuml;hrt.

Die Kommunikation l&auml;uft aber auch noch &uuml;ber die globalen Variablen `stacks`, `callbacks`, `canvas` und `src`.

In [1]:
# Game
ndisks = 4
stacks = []
callback = lambda event, data: print(event, data)


def notify(event, data):
    callback(event, data)


def new_game():
    stacks[:] = [list(range(ndisks))[::-1], [], []]
    notify('new_game', stacks)


def move_disk(i, j):
    x = stacks[i].pop()
    stacks[j].append(x)
    notify('update_stacks', stacks)

In [2]:
# View
from ipycanvas import Canvas


def draw_stack(stack, x):
    h = disk_height
    for i, disk in enumerate(stack):
        canvas.fill_style = colors[disk]
        w = disk_widths[disk]
        canvas.fill_rect(x - w/2, canvas.height-(i+1)*h, w, h)


def draw_stacks(stacks):
    canvas.clear()
    for x, stack in zip(positions, stacks):
        draw_stack(stack, x)


canvas_config = {
    'width': 300,
    'height': 100,
    'layout': {'border': '1px solid black'},
}

positions = [50, 150, 250]
disk_widths = [30, 50, 70, 90]
disk_height = 10
stack_width = 100
colors = ['brown', 'teal', 'blue', 'purple']

canvas = Canvas(**canvas_config)

In [3]:
# Controller
src = None

def on_key_down(key, *flags):
    global src
  
    if key in '123' and src is None:
        src = int(key) - 1
    elif key in '123' and src is not None:
        dst = int(key) - 1
        move_disk(src, dst)
        src = None
    elif key == 'n':
        new_game()
        src = None    
    else:
        src = None        

In [4]:
canvas.on_key_down(on_key_down)
canvas

Canvas(height=100, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…